In [42]:

import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt


In [5]:
df = pd.read_excel('/content/Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [8]:
#counting unique values in dataframe
count = df.nunique()
count

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
#separating credited transactions from our dataframe
df = df[~df["InvoiceNo"].str.contains("C", na = False)]

In [11]:
df['InviceNo'] = df['InvoiceNo'].astype('str')

<ipython-input-11-55e5d0cff24b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['InviceNo'] = df['InvoiceNo'].astype('str')


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532621 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    532621 non-null  object        
 1   StockCode    532621 non-null  object        
 2   Description  531167 non-null  object        
 3   Quantity     532621 non-null  int64         
 4   InvoiceDate  532621 non-null  datetime64[ns]
 5   UnitPrice    532621 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      532621 non-null  object        
 8   InviceNo     532621 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 40.6+ MB


In [23]:
Basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().reset_index().fillna(0).set_index('InvoiceNo')
Basket

,Description,Quantity
InvoiceNo,,
536365,CREAM CUPID HEARTS COAT HANGER,8
536365,GLASS STAR FROSTED T-LIGHT HOLDER,6
536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
536365,RED WOOLLY HOTTIE WHITE HEART.,6
536365,SET 7 BABUSHKA NESTING BOXES,2
...,...,...
581587,PLASTERS IN TIN STRONGMAN,12
581587,SPACEBOY LUNCH BOX,12
A563185,Adjust bad debt,1


In [29]:
#number of rows decrease as unstack changed the shape of datafram
#zero indicates that no product was bought by this invoicno
Basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
Basket

Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A563185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
#converting to 1 and 0 as it is required by our algorithm
#0 will mean present in transaction
#1 will mean item was not bought
def convert(x):
  if x<= 0:
    return 0
  if x>= 1:
    return 1
basket = Basket.applymap(convert)


**Metrics of Association Analysis**

**Support**--> if we have item A in 10 transactions out of 100 support is 10/100

**Confidence**--> We consider two items .If item B come in 5 transactions out of 10 transactions in which Item A was present the confidence will be  5/10

**Lift** --> Confidence of A and B divided by Support of B

In [39]:
#applying apriori algorithm
#By default, apriori returns the column indices of the items, which may be useful in downstream operations 
#such as association rule mining. For better readability, we can set use_colnames=True to convert these integer values into the respective item names:
#Lift basically tells us the likelihood of buying a the two items together 
#min_support  parameter is used to select the items with support values greater than the value specified by the parameter. 
#min_length parameter specifies the minimum number of items that you want in your rules.
itemset = apriori(basket, min_support=0.03,use_colnames = True)
frequent = association_rules(itemset, metric="lift", min_threshold=1)
frequent

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.047550,0.050995,0.031053,0.653061,12.806462,0.028628,2.735368
1,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.050995,0.047550,0.031053,0.608944,12.806462,0.028628,2.435585
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.049248,0.037166,0.030713,0.623645,16.779804,0.028883,2.558314
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051722,0.049248,0.037263,0.720450,14.629045,0.034716,3.401012
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.049248,0.051722,0.037263,0.756650,14.629045,0.034716,3.896768
6,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.101504,0.059098,0.040029,0.394359,6.673028,0.034030,1.553566
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.059098,0.101504,0.040029,0.677340,6.673028,0.034030,2.784651
8,(JUMBO BAG RED RETROSPOT),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.101504,0.057011,0.032994,0.325048,5.701477,0.027207,1.397119
9,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.057011,0.101504,0.032994,0.578723,5.701477,0.027207,2.132793


In [44]:
frequent.sort_values(by=['confidence'],ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.049248,0.051722,0.037263,0.756650,14.629045,0.034716,3.896768
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051722,0.049248,0.037263,0.720450,14.629045,0.034716,3.401012
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.059098,0.101504,0.040029,0.677340,6.673028,0.034030,2.784651
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.047550,0.050995,0.031053,0.653061,12.806462,0.028628,2.735368
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.049248,0.037166,0.030713,0.623645,16.779804,0.028883,2.558314
10,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.057448,0.101504,0.035129,0.611486,6.024253,0.029297,2.312650
1,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.050995,0.047550,0.031053,0.608944,12.806462,0.028628,2.435585
9,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.057011,0.101504,0.032994,0.578723,5.701477,0.027207,2.132793
12,(LUNCH BAG BLACK SKULL.),(LUNCH BAG RED RETROSPOT),0.061766,0.075885,0.031101,0.503535,6.635457,0.026414,1.861389
